In [1]:
import random
import requests
import os
from PIL import Image
import requests
from bs4 import BeautifulSoup
from lxml import html
import json

In [2]:
### Lấy danh sách các chapter của manga
def get_manga_chapters(id_manga):
    list_link=[]
    url = 'https://manhuaes.com/wp-admin/admin-ajax.php'
    payload = {
    'action': 'manga_get_chapters',
    'manga': f"{id_manga}"
    }

    res = requests.post(url, data=payload)
    soup=BeautifulSoup(res.content, 'html.parser')
    links=soup.find_all('a',href=True)
    for link in links:
        list_link.append(link['href'])

    return list_link

### Lấy các link ahr trong 1 chapter 
def get_img_chapter(link_chapter):
    list_img=[]
    res = requests.get(link_chapter)
    soup=BeautifulSoup(res.content,'html.parser')
    divs = soup.find_all('div', class_='page-break no-gaps')
    for div in divs:
        img = div.find('img')
        url = img.get('data-src')
        list_img.append(url)
    return list_img   

### Download và chuyển đổi định dạng ảnh bìa sang webp
def convert_img(link):
    try:
        name = str(random.randint(100000, 999999999))
        res = requests.get(link)
        with open('{}.jpg'.format(name), 'wb') as f:
            f.write(res.content)

        image = Image.open('{}.jpg'.format(name)).convert('RGB')
        image.save('{}.webp'.format(name), 'webp', optimize=True, quality=70)

        url = "https://img.mangalab.org/api/1/upload"
        payload = {'key': 'thankinhcac', 'format': 'json'}
        files = [('source', open('{}.webp'.format(name), 'rb'))]
        response = requests.post(url, data=payload, files=files)
        files[0][1].close()

        os.remove('{}.webp'.format(name))
        os.remove('{}.jpg'.format(name))
        response_data = response.json()
        image_url = response_data['image']['url']
        return image_url
    except (requests.RequestException, IOError) as e:
        # Handle request, file I/O errors
        print("An error occurred:", str(e))
        return None
    except Exception as e:
        # Handle other unexpected errors
        print("An unexpected error occurred:", str(e))
        return None

### Lấy các thông tin cơ bản của 1 manga 
def info_manga(manga_link):
    info=[]
    res=requests.get(manga_link)
    soup = BeautifulSoup(res.content, 'html.parser')
    lst_content=soup.find_all(class_='summary-content')
    link_tag = soup.find('link', rel='shortlink')
    id_manga = link_tag.get('href').split('=')[-1]
    info.append(id_manga)
    ratings=soup.find(class_='score font-meta total_votes').text
    info.append(soup.find('h1').text.strip().replace('\n', '').replace('\t', ''))
    if float(ratings)>1: 
        info.append(lst_content[2].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[3].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[4].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[5].text.strip().replace('\n', '').replace('\t', ''))
    else:
        info.append(lst_content[1].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[2].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[3].text.strip().replace('\n', '').replace('\t', ''))
        info.append(lst_content[4].text.strip().replace('\n', '').replace('\t', ''))


    img_manga=soup.find_all(class_='summary_image')[0].find('img')['data-src'].replace('-193x278','')
    
    info.append(convert_img(img_manga))
    return info


In [ ]:
import os
import mysql.connector

# Connect to the remote MySQL database
mydb = mysql.connector.connect(
  host="89.58.24.136",
  user="tsuhara",
  password="C7JHsZMDdCYjxD6Z",
  database="tsuhara"
)

In [3]:
info_manga('https://manhuaes.com/manga/apotheosis/')

['38182',
 'Apotheosis',
 'Apotheosis : Elevation to the status of a god Manhua / Apotheosis : Elevation to the status of a god Manhua ,Forged Success',
 'Ranzai Studio',
 'Updating',
 'Action, Fantasy, Martial Arts, Shounen',
 'https://img.thankinhcac.top/file/mangalab/02d857fdb530289c2e4822d4a463f626.webp']

In [4]:
mycursor = mydb.cursor()
sql = "INSERT INTO mc_comic (  name ,  yname ,  pic ,  picx ,   serialize ,  author ,   pic_author ,  txt_author ,  content,  addtime ) VALUES ( %s,%s,%s,%s,%s, %s,%s,%s,%s,%s)"
val=( 'rgtbs', 'rgtbs', '/attachment/comic/2023/05/13/202305131005052548.jpg', '/attachment/comic/2023/05/13/202305131005118699.jpg',  '连载', 'BotScans', 'gfb', 'fg', 'fg', 1683943682)
        
mycursor.execute(sql, val)

mydb.commit()

In [3]:
def get_all_manga():
    arr=[]
    for i in range(1,36):
        res=requests.get('https://manhuaes.com/page/{}/'.format(i))
        soup=BeautifulSoup(res.content,'html.parser')
        links=soup.find_all('h3',class_='h5')
        del res
        del soup
        for link in links:
            arr.append(link.find('a').get('href'))
    return arr


In [10]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

def get_all_manga():
    arr = []
    for i in range(1, 36):
        url = 'https://manhuaes.com/page/{}/'.format(i)
        retries = 1  # Number of times to retry the request
        for _ in range(retries):
            try:
                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'html.parser')
                links = soup.find_all('h3', class_='h5')
                for link in links:
                    arr.append(link.find('a').get('href'))
                break  # Break out of the retry loop if successful
            except RequestException as e:
                print(f"Error occurred: {str(e)}")
                print(f"Retrying request: {url}")
    return arr


In [11]:
a=get_all_manga()

In [13]:

# open file in write mode
with open(r'manga.txt', 'w') as fp:
    for item in a:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done
